###Modelo Random Forest Classifier para el dataset de Titanic

Dentro de este codigo se incorpora una solucion para crear un modelo tipo Random Forest para predecir si un pasajero sobrevive o no. A continuacion se explica el proceso por medio de comentarios y estrucutra de codigo para eficientar el aprendizaje del modelo y sus caracterisicas. La razon por la que se escogio este dataset fue por el motivo de que ya tenemos el ETL para analizar los datos de forma mas organizada y asi llevar a tener mejores resultados. Los datos trnsofmrados se encuentran el archivo train_data y train_results. Usaremos esta informacion para alimentar el modelo y finalmente usaremos tests para hacer pruebas.

La referencia para hacer este modelo se compoarte en la siguiente liga: https://machinelearningmastery.com/implement-random-forest-scratch-python/

Liga del dataset (No incluye el proceso de ETL): https://www.kaggle.com/competitions/titanic

Nombre: Rodolfo Sandoval
Matricula: A01720253

Nota: En caso de correr el codigo localmente asegurarse de tener el ruteo y la direccion correta de los archivos **train_data.csv** donde estan las caracteristicas y **split_survived.csv** donde esta el valor objetivo.

Descripcion de los datos:

Registros: 891

Numero de Caracteristicas: 6
Pclass, Age, Sex, Fam "Cantidad de familia", Fare, y Embarked.

Clases: 1 "Sobrevivio", 0 "No sobrevivio"

Se utilizaron estas caracteristicas ya que cada una influye y tiene un peso con de acuerdo a la correlacion con la clase de survived.

Vamos a estar usando varios decision trees. El objetivo es combinarlos y agregar las predicciones para tomar una decision final.

In [7]:
#Importacion de librerias
import numpy as np
import pandas as pd

#No es para el modelo sino para evaluarlo con metricas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix



# Funcion para dividir los datos en conjuntos de entrenamiento y prueba
def train_test_split(data, proporcion=0.2):
    np.random.shuffle(data)  #Mezaclamos los datos (data) con np.random.shuffle
    split_indice = int(len(data) * (1 - proporcion)) #Calculamos el indice en donde se dividen los datos de la proporcion (proporcion)
    train_data = data[:split_indice] #Datos desde el inicio hasta donde se hace la division
    test_data = data[split_indice:] #Datos desde la division hasta el final
    return train_data, test_data #Devolvemos el conjunto de los datos divididos

# Funcion para calcular la impureza Gini
def gini_impureza(etiquetas):
    clases_unicas, counts = np.unique(etiquetas, return_counts=True) #Encontramos las clases unicas
    clases_prob = counts / len(etiquetas) #Calculamos las probabilidades de las clases en relacion a la longitud total de etiquetas
    gini = 1 - np.sum(clases_prob ** 2) #Calculamos la impuresa de Gini (con esto determinamos como dividir los datos en los nodos del arbol)
    return gini

# Funcion para dividir los datos en funcion de un valor de umbral data = datos, feature_idx = indice donde se basa la division, y umbral = el valor umbral que se realiza para la division
def split_data(data, feature_idx, umbral):
    izq = data[data[:, feature_idx] <= umbral] #subconjunto con los valores sean menor o iguales al umbral
    derecha = data[data[:, feature_idx] > umbral] #subconjunto cuando sean mayores al umbral
    return izq, derecha

# Funcion para encontrar la mejor division para un conjunto de datos
def find_best_split(data):
    best_gini = float("inf") # Inicializar con un valor alto para comparacion
    best_feature_idx = None
    best_umbral = None

    num_features = data.shape[1] - 1 #Restamos la columna que incluye etiquetas

#Iteramos sobre las caracteristicas
    for feature_idx in range(num_features):
        unique_values = np.unique(data[:, feature_idx])
        for umbral in unique_values: #Sobre valores unicos
            izq, derecha = split_data(data, feature_idx, umbral)
            if len(izq) > 0 and len(derecha) > 0: # Dividir los datos en dos subconjuntos: izquierdo y derecho
                gini_izq = gini_impureza(izq[:, -1]) #Calcular impureza gini para lado izquierdo
                gini_derecha = gini_impureza(derecha[:, -1]) #Calcular impureza gini para lado derecho
                weighted_gini = (len(izq) / len(data)) * gini_izq + (len(derecha) / len(data)) * gini_derecha #Caclular la impureza despues de la division

                if weighted_gini < best_gini: #Checar si se encuentra una mejor division y replazar esos valores
                    best_gini = weighted_gini
                    best_feature_idx = feature_idx
                    best_umbral = umbral

    return best_feature_idx, best_umbral

# Funcion para construir un arbol de decision
def build_tree(data, max_depth):
    if max_depth == 0 or len(np.unique(data[:, -1])) == 1: #Checar si max_depth (profundidad de los nodos) es 0 o sin las etiquetas son iguales con unique
        clases_unicas, counts = np.unique(data[:, -1], return_counts=True)
        return clases_unicas[np.argmax(counts)]

    best_feature_idx, best_umbral = find_best_split(data)

    # Sacar la clase mayoritaria en caso de que no sea posible hacer una division (Clase con mayor cantidad de ejemplos)
    if best_feature_idx is None:
        clases_unicas, counts = np.unique(data[:, -1], return_counts=True)
        return clases_unicas[np.argmax(counts)]

    # Dividir los datos en dos subconjuntos usando la mejor caracteristica y umbral
    izq, derecha = split_data(data, best_feature_idx, best_umbral)
    izq_subtree = build_tree(izq, max_depth - 1)
    derecha_subtree = build_tree(derecha, max_depth - 1)
    return (best_feature_idx, best_umbral, izq_subtree, derecha_subtree) #Retornamos un nodo arbol con la mejor caracteristica

# Funcion para construir un Random Forest
def build_random_forest(data, num_arboles, max_depth):
    forest = [] #Lista de los arboles del bosque
    for _ in range(num_arboles):
        subset = np.random.choice(len(data), size=len(data), replace=True) #Creamos subconjunto aleatorio con replace para el conjunto de datos
        subset_data = data[subset]
        tree = build_tree(subset_data, max_depth) #Contruimos un arbol aleatorio
        forest.append(tree) #Agregamos el arbol al bosque
    return forest

# Funcion para hacer predicciones usando un arbol de decision
def predict(tree, sample):
    if isinstance(tree, np.int64):  # Si el nodo actual es una clase, devuelve la clase predicha
        return tree

    feature_idx, umbral, izq_subtree, derecha_subtree = tree

    if sample[feature_idx] <= umbral: # Comparamos el valor de la caracteristica (feature_idx) de la muestra con el umbral
        return predict(izq_subtree, sample) #Si es menor o igual al umbral nos vamos por el subarbol izq
    else:
        return predict(derecha_subtree, sample) #Si mayor al umbral nos vamos por el subarbol derecho

# Funcion para hacer predicciones con el Random Forest
def predict_random_forest(forest, sample):
    predi = [predict(tree, sample) for tree in forest]
    return np.bincount(predi).argmax()

# Cargar los datos de entrenamiento y resultados desde archivos CSV
train_data = pd.read_csv("train_data.csv")
train_results = pd.read_csv("split_survived.csv")

# Combinar los datos de entrenamiento y resultados en un unico DataFrame
combined_data = pd.merge(train_data, train_results, on='PassengerId')

# Convertir el DataFrame a un arreglo NumPy para facilitar su manejo
data_array = combined_data.values

train_data, test_data = train_test_split(data_array)
num_arboles = 100
max_depth = 3
forest = build_random_forest(train_data, num_arboles, max_depth)

# Hacer predicciones en datos de prueba
test_data = test_data[np.argsort(test_data[:, 0])]  # Ordenar por PassengerID
predi = [predict_random_forest(forest, sample) for sample in test_data[:, :-1]]
actual_etiquetas = test_data[:, -1]

# Crear un DataFrame con los resultados
results_df = pd.DataFrame({'PassengerID': test_data[:, 0], 'Prediccion': predi, 'Etiqueta Actual': actual_etiquetas})

# Ordenar el DataFrame por PassengerID
results_df = results_df.sort_values(by='PassengerID')

# Imprimir el DataFrame
print(results_df)

# Calcular la precision
accuracy = np.sum(predi == actual_etiquetas) / len(actual_etiquetas)
print("Precision del modelo:", accuracy)

#Creamos csv con los resultados del modelo
results_df.to_csv('model_results.csv', index=False)
print("Archivo CSV 'model_results.csv' guardado exitosamente.")

#Metricas
# Precisión
precision = precision_score(actual_etiquetas, predi)

# Exhaustividad (Recall)
recall = recall_score(actual_etiquetas, predi)

# F1-score
f1 = f1_score(actual_etiquetas, predi)

# Matriz de confusión
confusion = confusion_matrix(actual_etiquetas, predi)

# Exactitud (Accuracy)
accuracy = accuracy_score(actual_etiquetas, predi)

print("Precisión:", precision)
print("Exhaustividad (Recall):", recall)
print("F1-score:", f1)
print("Matriz de Confusión:")
print(confusion)
print("Exactitud (Accuracy):", accuracy)

     PassengerID  Prediccion  Etiqueta Actual
0              8           0                0
1             16           1                1
2             17           0                0
3             23           1                1
4             24           0                1
..           ...         ...              ...
174          866           1                1
175          870           1                1
176          871           0                0
177          875           1                1
178          884           0                0

[179 rows x 3 columns]
Precision del modelo: 0.8324022346368715
Archivo CSV 'model_results.csv' guardado exitosamente.
Precisión: 0.8205128205128205
Exhaustividad (Recall): 0.8
F1-score: 0.810126582278481
Matriz de Confusión:
[[85 14]
 [16 64]]
Exactitud (Accuracy): 0.8324022346368715
